# IMEX-RB: A Fast and Stable Time Integration Method

This notebook introduces and tests a novel time‐integration scheme, namely IMEX-RB. 
In the following, a linear system of ODEs stemming from a PDE space discretization is defined.
Then, the performance of IMEX-RB is compared with that of backward and forward Euler (BE and FE).

### Let us import the necessary time integration schemes

In [1]:
import numpy as np
from euler import backward_euler, forward_euler
from imexrb import imexrb

## Solve a 1D Heat equation

In [2]:
from problems1D import Heat1D

L = 1
Nx = 100
kappa = 0.1
bc_left = 0.0
bc_right = 0.0
t0 = 0.0
tf = 1.0
tspan = [t0, tf]
# Define the problem object
problem = Heat1D(Nx, L, kappa, bc_left, bc_right)
u0 = problem.initial_condition()


We now integrate over time, using different schemes. First of all, we define some parameters for the time integration

In [3]:
# Define number of timesteps
Nt = 100
# Define epsilon for absolute stability condition
epsilon = 1.0/np.linalg.cond(problem.A)
# Define max dimension of subspace before subiterations
maxsize = int(np.ceil(np.pow(Nx, 1/3)))
# Define max allowed enlargement during subiterations
maxsubiter = maxsize

In [4]:
# First, IMEX-RB
uIMEX, tvec = imexrb(problem, u0, tspan, Nt, epsilon, maxsize, maxsubiter)

NameError: name 'un' is not defined